## Pairs trading on different portfolios based on Machine Learning

Statistical arbitrage is the trading strategy developed using quantitative and statistical models to generate trading signals. Pairs-trading is one of the statistical arbitrage techniques. Here, this teschnique will be investigated using ML model LSTM (Long Short-term Memory) 

### Different steps in this project

1. Selecting 20 different stocks
2. Categorizing stocks into different types (aggressive or defensive types) 
3. Identifying pairs with co-integration test 
4. Constructing Portfolio
5. Forecasting stock prices using ML algorithm, LSTM 
6. Calculating Trading Profits 

### Why LSTM?

Among different types of models, deep learning models contain neural networks. Neural networks are capable of providing decent approximation to almost all non-linear functions. Due to the non-linearity of time series data, it is better to use neural networks to predict stock prices, rather than using regular linear frameworks.

Merits of using Neural networks
* Neural networks are designed to detect non-linearities that exist within the data
* Neural networks are scalable with reduced computational load, because, training or updating doesnot require retraining the entire model from scratch when new data is added. 

### Neural Networks

Neural Networks composed of input layer, several hiddien layers and then output layer. Input layer holds the input values, hidden layers process these input values through non-linear functions and the final output is provided at the output layer. Artificial Neurons are the building block of these different layers.

## Importing Libraries

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline  
import yfinance as yf 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() 

#### Selecting 20 different stocks

In [2]:
tickers = ['AMD','AAPL','CSCO','INTC','ORCL','MSFT','GOOGL','DIS','CEA','ERIC',
           'HPQ','WMT','PCG','JNJ','NEE','AEP','COKE','PEP','MCD','MRK']

start_date = '2008-01-01'
end_date = '2018-01-01'  

In [3]:
data_frames = {} 
fail = []
for tick in tickers:  
    df = pd.DataFrame()
    df = yf.download(tick,start=start_date,end=end_date)
    if len(df)==0:
        fail.append(tick) 
        continue 
    else:
        df.rename(columns={'Open':f'{tick}_Open','High':f'{tick}_High','Low':f'{tick}_Low','Close':f'{tick}_Close',
                       'Volume':f'{tick}_Volume','Adj Close':f'{tick}_Adj Close'},inplace=True) 
        data_frames[f'df_{tick}'] = df 

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['CEA']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*****

In [4]:
print(f'Total tickers considered : ',len(tickers)) 
print(f'Data download failed for tickers : ',fail) 
print(f'Total tickers available : ',len(data_frames))   

Total tickers considered :  20
Data download failed for tickers :  ['CEA']
Total tickers available :  19


In [5]:
data_frames['df_AMD'].shape 

(2518, 6)

#### Scaling the values in each dataframe and storing the corresponding arrays and data frames in dictionary format 

In [6]:
scaled_data = {}
scaled_df = {} 
for tick,df in data_frames.items():
    scaled_data[f'{tick}'] = sc.fit_transform(df) 
    scaled_df[f'{tick}'] = pd.DataFrame(data=scaled_data[f'{tick}'], columns=df.columns) 

#### Finding the standard deviation of each column in each dataframe and storing it in a dictionary

In [7]:
standard_dev = {}
for tick, df in scaled_df.items():
    standard_dev[f'{tick}'] = df.std() 

### Calculating $\beta$-coefficient

Beta measures the volatility, or systematic risk, of a stock in relation to the overall market. So, here we have to take S&P500 as benchmark.

In [8]:
df_bench_close = yf.download('^GSPC',start=start_date,end=end_date)['Adj Close'] 

[*********************100%%**********************]  1 of 1 completed


In [9]:
stock_data_close = {}

for tick, df in data_frames.items():    
    ticker = tick.split('_')[1] 
    stock_data_close[ticker] = df[f'{ticker}_Adj Close']  

In [10]:
benchmark_returns = df_bench_close.pct_change().dropna()
variance = benchmark_returns.var() 

In [11]:
stock_returns = {}
stock_cov = {} 
beta = {}

for tick in stock_data_close:
    stock_returns[tick] = stock_data_close[tick].pct_change().dropna()  # Calculating daily returns
    stock_cov[tick] = stock_returns[tick].cov(benchmark_returns)  # Calculating covariance
    beta[tick] = stock_cov[tick]/variance   # Calculating beta-coefficient 

#### $\beta$-coefficient for respective tickers

In [12]:
beta 

{'AMD': 1.4296052820603145,
 'AAPL': 0.948585563039305,
 'CSCO': 1.0460787518972827,
 'INTC': 1.0237096112657598,
 'ORCL': 0.9937152094176303,
 'MSFT': 0.9565029923131274,
 'GOOGL': 0.936107159748868,
 'DIS': 1.0621765244262076,
 'ERIC': 1.242361915981989,
 'HPQ': 0.9722097121061177,
 'WMT': 0.5068536501811887,
 'PCG': 0.5456102914303368,
 'JNJ': 0.5622697254528666,
 'NEE': 0.686813783457772,
 'AEP': 0.6335410580558766,
 'COKE': 0.7010562183233562,
 'PEP': 0.5271189510910477,
 'MCD': 0.5729175841412574,
 'MRK': 0.7842098774320623}

In [21]:
sorted_beta = sorted(beta.items(), key=lambda x: x[1], reverse=True)
beta_df = pd.DataFrame(sorted_beta, columns=[['Ticker','Beta values']])
beta_df 

,Ticker,Beta values
0,AMD,1.429605
1,ERIC,1.242362
2,DIS,1.062177
3,CSCO,1.046079
4,INTC,1.023710
5,ORCL,0.993715
6,HPQ,0.972210
7,MSFT,0.956503
8,AAPL,0.948586
9,GOOGL,0.936107


Value of $\beta$ is always +ve. It is either above or below 1. 

* If $\beta<1 \implies$ variability of this asset's return rate is lower than market portfolio
* If $\beta>1 \implies$ variability of this asset's return rate is higher than market portfolio

### Classifying stocks as Aggressive and Defensive

Categorizing stocks as aggressive and defensive is based on their beta value

* Aggressive stocks : Stocks facing higher volatility in revenue, leading to higher stock returns and sensitivity to the market index. High-tech stocks are regarded as aggressive. Also, numerically, if $\beta>1$, it is referred to as aggressive stocks.

* Defensive stocks : These kind of stocks can be found under daily-necessity sectors, sucha as public utilities, pharmaceutical firms, airline and tourism industries etc... If $\beta<1$, it is referred to as defensive stocks.

In [14]:
aggressive = []
defensive = []

for tick, value in beta.items():
    if value>1.0:
        aggressive.append(tick)
    else:
        defensive.append(tick)     

In [15]:
aggressive 

['AMD', 'CSCO', 'INTC', 'DIS', 'ERIC']

In [16]:
defensive 

['AAPL',
 'ORCL',
 'MSFT',
 'GOOGL',
 'HPQ',
 'WMT',
 'PCG',
 'JNJ',
 'NEE',
 'AEP',
 'COKE',
 'PEP',
 'MCD',
 'MRK']

### Identifying pairs with cointegration test

1. Augmented Dickey-Fuller unit root test : Used to test the cointegration relationship between two objects after conducting stationary test
2. Augmented Engle-Granger two step cointegration test

#### Stationarity and Augmented Dickey-Fuller test (ADF)

Stationarity in a time series analysis implies that the statistical properties of a time series such as mean, variance and auto-correlation are constant over time. ADF is a commonly used statistical test to check for stationarity. The null hypothesis of ADF test is that the time series has a unit root, that is it is non-stationary whereas alternative hypothesis is that the time series is stationary.

* p-value $<0.05$ gives that the null hypothesis can be rejected and the series is stationary
* p-value $>0.05$ gives that the series is non-stationary  

In [22]:
from statsmodels.tsa.stattools import adfuller 

#### Function to perform adf test

In [33]:
def adf_test(series, tick):
    result = adfuller(series)
    print('-'*20,tick,'-'*20)
    print()  
    print(f'ADF statistic for {tick} : {result[0]}')
    print(f'p-value for {tick} : {result[1]}') 
    for key, values in result[4].items():
        print(f'Critical values {key} for {tick} : {values}') 
    print()

In [34]:
for tick, df in stock_data_close.items():
    adf_test(df,tick) 

-------------------- AMD --------------------

ADF statistic for AMD : -1.4140006569239492
p-value for AMD : 0.5755459280644902
Critical values 1% for AMD : -3.4329610922579095
Critical values 5% for AMD : -2.8626935681060375
Critical values 10% for AMD : -2.567384088736619

-------------------- AAPL --------------------

ADF statistic for AAPL : 0.8142216991646601
p-value for AAPL : 0.9918622989293024
Critical values 1% for AAPL : -3.4329507078222634
Critical values 5% for AAPL : -2.8626889823128554
Critical values 10% for AAPL : -2.567381647203466

-------------------- CSCO --------------------

ADF statistic for CSCO : 0.0715257751880899
p-value for CSCO : 0.9640623878923115
Critical values 1% for CSCO : -3.4329517425474014
Critical values 5% for CSCO : -2.862689439250822
Critical values 10% for CSCO : -2.5673818904827863

-------------------- INTC --------------------

ADF statistic for INTC : 0.546356512465547
p-value for INTC : 0.9862160031409304
Critical values 1% for INTC : -3.

#### Creating a data frame with Adj Close values

In [56]:
def dataframeClose(tickers):
    df_close = pd.DataFrame()

    for tick in tickers:
        df = yf.download(tick, start=start_date, end=end_date)
        if len(df)==0:
           continue
        else:
          df_close[tick]=df['Adj Close'] 

    return df_close       

In [57]:
df_aggClose = dataframeClose(aggressive) 

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [58]:
df_defClose = dataframeClose(defensive) 

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [49]:
def p_valueFn(series):
    result = adfuller(series)
    return result[1] 

In [46]:
import itertools 

In [61]:
def pairsList(df_close):
    pairs_l = []
    pairs_g = []

    for pair in itertools.combinations(df_close.columns,2):
        spread  = df_close[pair[0]] - df_close[pair[1]]
        p_value = p_valueFn(spread.dropna()) 
        if p_value<0.05:
            pairs_l.append((pair[0],pair[1],p_value))
        else:
            pairs_g.append((pair[0],pair[1],p_value)) 
    return pairs_l, pairs_g            

In [62]:
pairs_aggL, pairs_aggG = pairsList(df_aggClose)
pairs_defL, pairs_defG = pairsList(df_defClose) 

In [63]:
pairs_aggG 

[('AMD', 'CSCO', 0.7575658162909402),
 ('AMD', 'INTC', 0.8713820272100768),
 ('AMD', 'DIS', 0.9082338133162171),
 ('AMD', 'ERIC', 0.6689592910809128),
 ('CSCO', 'INTC', 0.3109244557942084),
 ('CSCO', 'DIS', 0.852396026093164),
 ('CSCO', 'ERIC', 0.9925576006592457),
 ('INTC', 'DIS', 0.8110223483145513),
 ('INTC', 'ERIC', 0.9957810217995531),
 ('DIS', 'ERIC', 0.9438495897191578)]

In [64]:
pairs_aggL 

[]

In [65]:
pairs_defG 

[('AAPL', 'ORCL', 0.08049372591711967),
 ('AAPL', 'MSFT', 0.990322914954289),
 ('AAPL', 'GOOGL', 0.2994459166166149),
 ('AAPL', 'HPQ', 0.8977018799976404),
 ('AAPL', 'WMT', 0.8525849486490338),
 ('AAPL', 'PCG', 0.514151123870793),
 ('AAPL', 'JNJ', 0.9516738106608016),
 ('AAPL', 'NEE', 0.3175499168272615),
 ('AAPL', 'AEP', 0.1916787772103808),
 ('AAPL', 'COKE', 0.9638504195463066),
 ('AAPL', 'PEP', 0.7898455623551851),
 ('AAPL', 'MCD', 0.9948419881856896),
 ('AAPL', 'MRK', 0.05583430444904344),
 ('ORCL', 'MSFT', 0.995492367219467),
 ('ORCL', 'GOOGL', 0.6968372189026115),
 ('ORCL', 'HPQ', 0.5786386301689384),
 ('ORCL', 'WMT', 0.24689940251091602),
 ('ORCL', 'PCG', 0.3594201297997637),
 ('ORCL', 'JNJ', 0.9876643077793417),
 ('ORCL', 'AEP', 0.5090698699722197),
 ('ORCL', 'COKE', 0.9763368670473037),
 ('ORCL', 'PEP', 0.9345251262029588),
 ('ORCL', 'MCD', 0.9985675093626746),
 ('MSFT', 'GOOGL', 0.9677535843832012),
 ('MSFT', 'HPQ', 0.9990245837544638),
 ('MSFT', 'WMT', 0.9975022700639917),
 

In [66]:
pairs_defL

[('ORCL', 'NEE', 0.028479283242000122),
 ('ORCL', 'MRK', 0.02143322681061044),
 ('GOOGL', 'AEP', 0.0014554417596749126)]

In [67]:
df_defClose.head() 

,AAPL,ORCL,MSFT,GOOGL,HPQ,WMT,PCG,JNJ,NEE,AEP,COKE,PEP,MCD,MRK
Date,,,,,,,,,,,,,,
2008-01-02,5.883143,18.300968,25.535206,17.127239,15.035050,10.940400,29.559351,40.545528,10.177975,23.933054,48.633270,46.554630,36.491596,30.541338
2008-01-03,5.885862,18.805479,25.643955,17.130739,15.035050,10.819100,29.655041,40.557823,10.273086,24.005421,47.375198,46.857616,36.384834,30.514711
2008-01-04,5.436562,17.926653,24.926188,16.422592,14.193199,10.665139,29.627693,40.502449,10.556870,23.927889,46.183380,46.715385,35.832104,30.264511
2008-01-07,5.363794,18.105677,25.092943,16.228872,13.732921,10.861087,30.119785,41.129913,11.049279,24.625723,45.595749,47.809845,36.447632,30.834127
2008-01-08,5.170848,17.210564,24.251921,15.789686,13.078825,10.723456,29.935257,41.179138,10.975646,24.610212,45.446770,48.193214,35.850964,31.760435


In [68]:
import statsmodels.api as sm 

In [69]:
coint_result = sm.tsa.stattools.coint(df_defClose['JNJ'], df_defClose['NEE'])

# Print the results
coint_t, p_value, critical_values = coint_result
print(f'Cointegration Test Statistic: {coint_t}')
print(f'p-value: {p_value}')
print(f'Critical Values: {critical_values}') 

Cointegration Test Statistic: -5.060284462506643
p-value: 0.00012736170808537407
Critical Values: [-3.90079646 -3.33855861 -3.04613545]
